Completed:
- Extracted data at intervals of first record each day
- Filtered data from tank NDS002
- Created calculation model for NPK ratio
- Push Code to GitHub

Remaining Task:
- Categorize into levels
- Model to extract time taken to reach ‘Optimal’ level
- Automate NPK calculation model for future data
- Test on increased size of dataset

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
'''import psycopg2
import pandas as pd

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="smart_composting_api",
    user="npds_2025",
    password="npds_2025_eh0atNiA5MVx2FYY3UnqVo5Vzv_0N9MRnSZ_3dkJgT_r2EIONEpFzV1o3IXHSFsjUX8hXT-9OgKqt8f512RPWJohKdM_pA-dAfimfXXOuke5C0Z9irOt4GrEV5R",
    host="db.composting.tinkerthings.global",
    port="6969"
)

# Define Custom SQL Query from PgAdmin
# Extract Sensor Data from tank NDS002 only
query = """
WITH RankedData AS (
  SELECT 
    SD.dataid, 
    D.devicename, 
    S.sensor, 
    S.sensorid, 
    SD.value, 
    DD.dbtimestamp,
    ROW_NUMBER() OVER (
      PARTITION BY 
        D.devicename, 
        S.sensorid, 
        DATE(DD.dbtimestamp)
      ORDER BY 
        DD.dbtimestamp ASC
    ) AS rn
  FROM public.sensordata AS SD
  JOIN public.devicedata   DD ON SD.dataid   = DD.dataid
  JOIN public.sensors      S  ON S.sensorid  = SD.sensorid
  JOIN public.devices      D  ON D.deviceid  = DD.deviceid
  WHERE D.devicedescription LIKE 'NP DS 2024 Group A%'
  OR D.devicedescription LIKE 'NP DS 2024 Group D%'
  OR D.devicedescription LIKE 'NP DS 2024 Group E%'
  OR D.devicename = 'NDS006'
  OR D.devicename = 'NDS011'
  OR D.devicename = 'NDS012'
  OR D.devicename = 'NDS013'
  OR D.devicename = 'NDS015'
)
SELECT 
  dataid, 
  devicename, 
  sensor, 
  sensorid, 
  value, 
  dbtimestamp
FROM RankedData
WHERE rn = 1
ORDER BY dbtimestamp DESC;
"""

# Load result into DataFrame
chunks = pd.read_sql_query(query, conn, chunksize=1000)

# Combine or process each chunk
df_list = []
for chunk in chunks:
    print(f"Loaded chunk with {len(chunk)} rows.")
    df_list.append(chunk)  # Or process chunk directly

# Combine all chunks into a full DataFrame
df = pd.concat(df_list, ignore_index=True)
print(f"Total rows loaded: {len(df)}")

# Close connection
conn.close()'''

C:\Users\leong\AppData\Local\Temp\ipykernel_28680\2180147733.py:58: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chunks = pd.read_sql_query(query, conn, chunksize=1000)


Loaded chunk with 1000 rows.
Loaded chunk with 1000 rows.
Loaded chunk with 1000 rows.
Loaded chunk with 1000 rows.
Loaded chunk with 1000 rows.
Loaded chunk with 1000 rows.
Loaded chunk with 1000 rows.
Loaded chunk with 1000 rows.
Loaded chunk with 1000 rows.
Loaded chunk with 1000 rows.
Loaded chunk with 923 rows.
Total rows loaded: 10923


In [ ]:
# To avoid constantly pulling from pgAdmin
'''df.to_csv("Raw Data.csv ", index=False)'''

In [4]:
# load data
df = pd.read_csv('Raw Data.csv')

In [5]:
df.head(10)

,dataid,devicename,sensor,sensorid,value,dbtimestamp
0,71446705,NDS016,Soil Potassium,14,52.0,2025-05-30 00:01:55.980744
1,71446699,NDS016,Soil Moisture,9,18.9,2025-05-30 00:01:55.980744
2,71446702,NDS016,Soil pH,10,5.5,2025-05-30 00:01:55.980744
3,71446701,NDS016,Soil EC,11,213.0,2025-05-30 00:01:55.980744
4,71446703,NDS016,Soil Nitrogen,12,6.0,2025-05-30 00:01:55.980744
5,71446704,NDS016,Soil Phosphorus,13,59.0,2025-05-30 00:01:55.980744
6,71446700,NDS016,Soil Temperature,8,25.6,2025-05-30 00:01:55.980744
7,71406569,NDS016,Soil Nitrogen,12,11.0,2025-05-29 00:01:26.817793
8,71406566,NDS016,Soil Temperature,8,27.8,2025-05-29 00:01:26.817793
9,71406571,NDS016,Soil Potassium,14,65.0,2025-05-29 00:01:26.817793


In [6]:
df['dbtimestamp'] = df['dbtimestamp'].astype('datetime64[ns]')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10923 entries, 0 to 10922
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   dataid       10923 non-null  int64         
 1   devicename   10923 non-null  object        
 2   sensor       10923 non-null  object        
 3   sensorid     10923 non-null  int64         
 4   value        10923 non-null  float64       
 5   dbtimestamp  10923 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 512.1+ KB


Filter out 1970

In [8]:
# Filter out rows where the year is 2017
filtered_df = df[df["dbtimestamp"].dt.year != 1970]

filtered_df.tail()

,dataid,devicename,sensor,sensorid,value,dbtimestamp
10918,381388,NDS016,Soil Nitrogen,12,255.0,2024-07-18 11:50:40.977031
10919,381387,NDS016,Soil pH,10,0.0,2024-07-18 11:50:40.968742
10920,381386,NDS016,Soil EC,11,0.0,2024-07-18 11:50:40.964882
10921,381385,NDS016,Soil Temperature,8,1408.0,2024-07-18 11:50:40.961036
10922,381384,NDS016,Soil Moisture,9,1152.0,2024-07-18 11:50:40.952504


Feature Engineering: Number of Days since First Recording

Re-Edit to do by group not by full dataset

In [9]:
filtered_df['dbtimestamp'] = filtered_df['dbtimestamp'].astype('datetime64[ns]')

In [10]:
filtered_df['days_from_start'] = (filtered_df['dbtimestamp'] - filtered_df['dbtimestamp'].min()).dt.days

In [11]:
filtered_df.head()

,dataid,devicename,sensor,sensorid,value,dbtimestamp,days_from_start
0,71446705,NDS016,Soil Potassium,14,52.0,2025-05-30 00:01:55.980744,315
1,71446699,NDS016,Soil Moisture,9,18.9,2025-05-30 00:01:55.980744,315
2,71446702,NDS016,Soil pH,10,5.5,2025-05-30 00:01:55.980744,315
3,71446701,NDS016,Soil EC,11,213.0,2025-05-30 00:01:55.980744,315
4,71446703,NDS016,Soil Nitrogen,12,6.0,2025-05-30 00:01:55.980744,315


Format dataframe so each sensor becomes column and value is the cell content

In [12]:
# Pivot with both dbtimestamp and devicename in the index
npk_row = filtered_df.pivot(index=['dbtimestamp', 'devicename'], columns='sensor', values='value').reset_index()

# Sort by dbtimestamp
npk_row = npk_row.sort_values(by='dbtimestamp', ascending=False).reset_index(drop=True)

# Display result
npk_row.head()

sensor,dbtimestamp,devicename,Air Temperature,CO2,Humidity,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH
0,2025-05-30 00:01:55.980744,NDS016,NaN,NaN,NaN,213.0,18.9,6.0,59.0,52.0,25.6,5.5
1,2025-05-29 00:01:26.817793,NDS016,NaN,NaN,NaN,240.0,18.9,11.0,72.0,65.0,27.8,5.5
2,2025-05-28 00:00:48.939404,NDS016,NaN,NaN,NaN,247.0,18.1,13.0,76.0,68.0,26.5,5.5
3,2025-05-27 00:00:19.025437,NDS016,NaN,NaN,NaN,270.0,19.2,17.0,87.0,80.0,27.3,5.5
4,2025-05-26 00:01:19.007408,NDS016,NaN,NaN,NaN,292.0,20.7,22.0,98.0,90.0,26.9,5.2


Categorize into Numerical Columns

In [13]:
import numpy as np
import pandas as pd

num_cols = [
    n for n in npk_row.columns
    if npk_row[n].dtypes != 'O' and not np.issubdtype(npk_row[n].dtype, np.datetime64)
]

df_num = npk_row[num_cols]

df_num.head()


sensor,Air Temperature,CO2,Humidity,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH
0,NaN,NaN,NaN,213.0,18.9,6.0,59.0,52.0,25.6,5.5
1,NaN,NaN,NaN,240.0,18.9,11.0,72.0,65.0,27.8,5.5
2,NaN,NaN,NaN,247.0,18.1,13.0,76.0,68.0,26.5,5.5
3,NaN,NaN,NaN,270.0,19.2,17.0,87.0,80.0,27.3,5.5
4,NaN,NaN,NaN,292.0,20.7,22.0,98.0,90.0,26.9,5.2


Null Imputation: Iterative Imputer

In [14]:
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer

# Apply Iterative Imputer
imp = IterativeImputer(max_iter=10, random_state=0)
npk_row[num_cols] = imp.fit_transform(npk_row[num_cols])

c:\Users\leong\anaconda\Lib\site-packages\sklearn\impute\_iterative.py:801: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [15]:
print(npk_row.isnull().mean().sort_values(ascending = False))

sensor
dbtimestamp         0.0
devicename          0.0
Air Temperature     0.0
CO2                 0.0
Humidity            0.0
Soil EC             0.0
Soil Moisture       0.0
Soil Nitrogen       0.0
Soil Phosphorus     0.0
Soil Potassium      0.0
Soil Temperature    0.0
Soil pH             0.0
dtype: float64


In [16]:
npk_row.head()

sensor,dbtimestamp,devicename,Air Temperature,CO2,Humidity,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH
0,2025-05-30 00:01:55.980744,NDS016,8.848828e+06,-1.485710e+07,-463675.453204,213.0,18.9,6.0,59.0,52.0,25.6,5.5
1,2025-05-29 00:01:26.817793,NDS016,8.820416e+06,-1.520294e+07,-461140.799171,240.0,18.9,11.0,72.0,65.0,27.8,5.5
2,2025-05-28 00:00:48.939404,NDS016,8.816296e+06,-1.525375e+07,-460693.965770,247.0,18.1,13.0,76.0,68.0,26.5,5.5
3,2025-05-27 00:00:19.025437,NDS016,8.793050e+06,-1.553598e+07,-458680.914135,270.0,19.2,17.0,87.0,80.0,27.3,5.5
4,2025-05-26 00:01:19.007408,NDS016,8.774489e+06,-1.576099e+07,-457093.155473,292.0,20.7,22.0,98.0,90.0,26.9,5.2


Create Calculation Method for NPK:
- Automate for future data

In [17]:
def calculate_npk_ratio(row):
    N = row['Soil Nitrogen']
    P = row['Soil Phosphorus']
    K = row['Soil Potassium']

    # Avoid division by zero
    if min(N, P, K) == 0:
        return float('nan')

    # Normalize based on smallest value
    min_val = min(N, P, K)
    return f"{round(N/min_val, 1)} : {round(P/min_val, 1)} : {round(K/min_val, 1)}"

In [18]:
# Create new column
npk_row['NPK Ratio'] = npk_row.apply(calculate_npk_ratio, axis=1)

# View result
npk_row.head()

sensor,dbtimestamp,devicename,Air Temperature,CO2,Humidity,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH,NPK Ratio
0,2025-05-30 00:01:55.980744,NDS016,8.848828e+06,-1.485710e+07,-463675.453204,213.0,18.9,6.0,59.0,52.0,25.6,5.5,1.0 : 9.8 : 8.7
1,2025-05-29 00:01:26.817793,NDS016,8.820416e+06,-1.520294e+07,-461140.799171,240.0,18.9,11.0,72.0,65.0,27.8,5.5,1.0 : 6.5 : 5.9
2,2025-05-28 00:00:48.939404,NDS016,8.816296e+06,-1.525375e+07,-460693.965770,247.0,18.1,13.0,76.0,68.0,26.5,5.5,1.0 : 5.8 : 5.2
3,2025-05-27 00:00:19.025437,NDS016,8.793050e+06,-1.553598e+07,-458680.914135,270.0,19.2,17.0,87.0,80.0,27.3,5.5,1.0 : 5.1 : 4.7
4,2025-05-26 00:01:19.007408,NDS016,8.774489e+06,-1.576099e+07,-457093.155473,292.0,20.7,22.0,98.0,90.0,26.9,5.2,1.0 : 4.5 : 4.1


Categorize into 'Low', 'High', 'Optimal'
- Long Bean: 1:2:2
- Spinach: 2:1:1
- Lady Finger 1:1:1

In [19]:
def nitrogen(value):
    if value <= 100:
        return 'Low'
    elif 100 < value <= 300:
        return 'High'
    else:
        return 'Optimal'

In [20]:
def pp(value):
    if value <= 300:
        return 'Low'
    elif 300 < value <= 600:
        return 'High'
    else:
        return 'Optimal'

In [21]:
npk_row['N Category'] = npk_row['Soil Nitrogen'].apply(nitrogen)
npk_row['P Category'] = npk_row['Soil Phosphorus'].apply(pp)
npk_row['K Category'] = npk_row['Soil Potassium'].apply(pp)

# View result
npk_row.head()

sensor,dbtimestamp,devicename,Air Temperature,CO2,Humidity,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH,NPK Ratio,N Category,P Category,K Category
0,2025-05-30 00:01:55.980744,NDS016,8.848828e+06,-1.485710e+07,-463675.453204,213.0,18.9,6.0,59.0,52.0,25.6,5.5,1.0 : 9.8 : 8.7,Low,Low,Low
1,2025-05-29 00:01:26.817793,NDS016,8.820416e+06,-1.520294e+07,-461140.799171,240.0,18.9,11.0,72.0,65.0,27.8,5.5,1.0 : 6.5 : 5.9,Low,Low,Low
2,2025-05-28 00:00:48.939404,NDS016,8.816296e+06,-1.525375e+07,-460693.965770,247.0,18.1,13.0,76.0,68.0,26.5,5.5,1.0 : 5.8 : 5.2,Low,Low,Low
3,2025-05-27 00:00:19.025437,NDS016,8.793050e+06,-1.553598e+07,-458680.914135,270.0,19.2,17.0,87.0,80.0,27.3,5.5,1.0 : 5.1 : 4.7,Low,Low,Low
4,2025-05-26 00:01:19.007408,NDS016,8.774489e+06,-1.576099e+07,-457093.155473,292.0,20.7,22.0,98.0,90.0,26.9,5.2,1.0 : 4.5 : 4.1,Low,Low,Low


In [22]:
npk_row.tail()

sensor,dbtimestamp,devicename,Air Temperature,CO2,Humidity,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH,NPK Ratio,N Category,P Category,K Category
5703,2024-07-18 11:50:40.977031,NDS016,-1.259112e+05,-1.242424e+08,309003.527625,1459.147346,-362.906066,255.000000,1133.897069,-1246.297953,2565.633838,386.392990,-0.2 : -0.9 : 1.0,High,Optimal,Low
5704,2024-07-18 11:50:40.968742,NDS016,4.571207e+06,-6.475201e+07,-133605.067573,34048.707195,9463.763402,1191.480434,7000.579283,384.383471,5444.329412,0.000000,3.1 : 18.2 : 1.0,Optimal,Optimal,High
5705,2024-07-18 11:50:40.964882,NDS016,4.990964e+06,-6.013408e+07,-159871.364775,0.000000,1127.352688,616.589779,5215.266747,-12263.879367,7611.326456,374.680628,-0.1 : -0.4 : 1.0,Optimal,Optimal,Low
5706,2024-07-18 11:50:40.961036,NDS016,-1.700256e+06,-1.419787e+08,415084.896444,-27342.621040,-8569.427010,-1377.910052,4485.987470,1980.079211,1408.000000,551.699944,1.0 : -3.3 : -1.4,Low,Optimal,Optimal
5707,2024-07-18 11:50:40.952504,NDS016,-6.340074e+06,-1.974394e+08,797167.745253,16852.150828,1152.000000,9364.266314,7798.828169,28983.680919,-8293.382747,381.739247,1.2 : 1.0 : 3.7,Optimal,Optimal,Optimal


In [23]:
# Long Bean: 1:2:2
# Spinach: 2:1:1
# Lady Finger 1:1:1

def categorize_npk(row):
    ratio = row['NPK Ratio']

    if ratio == '1.0 : 2.0 : 2.0':
        return 'Long Bean'
    elif ratio == '2.0 : 1.0 : 1.0':
        return 'Spinach'
    elif ratio == '1.0 : 1.0 : 1.0':
        return 'Lady Finger'
    else:
        return 'Invalid'

In [24]:
npk_row['NPK Level'] = npk_row.apply(categorize_npk, axis=1)

# View result
npk_row.head()

sensor,dbtimestamp,devicename,Air Temperature,CO2,Humidity,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH,NPK Ratio,N Category,P Category,K Category,NPK Level
0,2025-05-30 00:01:55.980744,NDS016,8.848828e+06,-1.485710e+07,-463675.453204,213.0,18.9,6.0,59.0,52.0,25.6,5.5,1.0 : 9.8 : 8.7,Low,Low,Low,Invalid
1,2025-05-29 00:01:26.817793,NDS016,8.820416e+06,-1.520294e+07,-461140.799171,240.0,18.9,11.0,72.0,65.0,27.8,5.5,1.0 : 6.5 : 5.9,Low,Low,Low,Invalid
2,2025-05-28 00:00:48.939404,NDS016,8.816296e+06,-1.525375e+07,-460693.965770,247.0,18.1,13.0,76.0,68.0,26.5,5.5,1.0 : 5.8 : 5.2,Low,Low,Low,Invalid
3,2025-05-27 00:00:19.025437,NDS016,8.793050e+06,-1.553598e+07,-458680.914135,270.0,19.2,17.0,87.0,80.0,27.3,5.5,1.0 : 5.1 : 4.7,Low,Low,Low,Invalid
4,2025-05-26 00:01:19.007408,NDS016,8.774489e+06,-1.576099e+07,-457093.155473,292.0,20.7,22.0,98.0,90.0,26.9,5.2,1.0 : 4.5 : 4.1,Low,Low,Low,Invalid


In [25]:
def npk_done(row):
    ratio = row['NPK Ratio']
    n = row['N Category']
    p = row['P Category']
    k = row['K Category']

    if ratio == '1.0 : 2.0 : 2.0' and n == 'Optimal' and p == 'Optimal' and k=='Optimal':
        return 'Long Bean Done'
    elif ratio == '2.0 : 1.0 : 1.0' and n == 'Optimal' and p == 'Optimal' and k=='Optimal':
        return 'Spinach Done'
    elif ratio == '1.0 : 1.0 : 1.0' and n == 'Optimal' and p == 'Optimal' and k=='Optimal':
        return 'Lady Finger Done'
    elif n == 'Optimal' and p == 'Optimal' and k=='Optimal':
        return 'Optimal NPK'
    else:
        return 'Not Done Composting'

In [26]:
npk_row['NPK Done'] = npk_row.apply(npk_done, axis=1)

# View result
npk_row.head()

sensor,dbtimestamp,devicename,Air Temperature,CO2,Humidity,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH,NPK Ratio,N Category,P Category,K Category,NPK Level,NPK Done
0,2025-05-30 00:01:55.980744,NDS016,8.848828e+06,-1.485710e+07,-463675.453204,213.0,18.9,6.0,59.0,52.0,25.6,5.5,1.0 : 9.8 : 8.7,Low,Low,Low,Invalid,Not Done Composting
1,2025-05-29 00:01:26.817793,NDS016,8.820416e+06,-1.520294e+07,-461140.799171,240.0,18.9,11.0,72.0,65.0,27.8,5.5,1.0 : 6.5 : 5.9,Low,Low,Low,Invalid,Not Done Composting
2,2025-05-28 00:00:48.939404,NDS016,8.816296e+06,-1.525375e+07,-460693.965770,247.0,18.1,13.0,76.0,68.0,26.5,5.5,1.0 : 5.8 : 5.2,Low,Low,Low,Invalid,Not Done Composting
3,2025-05-27 00:00:19.025437,NDS016,8.793050e+06,-1.553598e+07,-458680.914135,270.0,19.2,17.0,87.0,80.0,27.3,5.5,1.0 : 5.1 : 4.7,Low,Low,Low,Invalid,Not Done Composting
4,2025-05-26 00:01:19.007408,NDS016,8.774489e+06,-1.576099e+07,-457093.155473,292.0,20.7,22.0,98.0,90.0,26.9,5.2,1.0 : 4.5 : 4.1,Low,Low,Low,Invalid,Not Done Composting


In [27]:
extract = npk_row.copy()

In [28]:
extract.count()

sensor
dbtimestamp         5708
devicename          5708
Air Temperature     5708
CO2                 5708
Humidity            5708
Soil EC             5708
Soil Moisture       5708
Soil Nitrogen       5708
Soil Phosphorus     5708
Soil Potassium      5708
Soil Temperature    5708
Soil pH             5708
NPK Ratio           5617
N Category          5708
P Category          5708
K Category          5708
NPK Level           5708
NPK Done            5708
dtype: int64

In [29]:
# Filter rows with null values
rows_with_nulls = extract[extract.isnull().any(axis=1)]

# Print rows with null values
rows_with_nulls.head()

sensor,dbtimestamp,devicename,Air Temperature,CO2,Humidity,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH,NPK Ratio,N Category,P Category,K Category,NPK Level,NPK Done
1840,2024-10-08 00:00:04.936801,NDS015,1.060776e+06,-1.078419e+08,170718.330044,31026.288788,8405.823548,4869.244294,5984.438368,0.0,1473.519665,585.968309,NaN,Optimal,Optimal,Low,Invalid,Not Done Composting
1861,2024-10-07 00:00:14.899648,NDS015,1.060776e+06,-1.078419e+08,170718.330044,31026.288788,8405.823548,4869.244294,5984.438368,0.0,1473.519665,585.968309,NaN,Optimal,Optimal,Low,Invalid,Not Done Composting
1977,2024-10-05 00:00:29.303118,NDS015,1.060776e+06,-1.078419e+08,170718.330044,31026.288788,8405.823548,4869.244294,5984.438368,0.0,1473.519665,585.968309,NaN,Optimal,Optimal,Low,Invalid,Not Done Composting
2088,2024-10-04 00:00:02.253291,NDS015,1.060776e+06,-1.078419e+08,170718.330044,31026.288788,8405.823548,4869.244294,5984.438368,0.0,1473.519665,585.968309,NaN,Optimal,Optimal,Low,Invalid,Not Done Composting
2136,2024-10-03 00:00:07.487730,NDS015,1.060776e+06,-1.078419e+08,170718.330044,31026.288788,8405.823548,4869.244294,5984.438368,0.0,1473.519665,585.968309,NaN,Optimal,Optimal,Low,Invalid,Not Done Composting


In [30]:
# List of columns to exclude
exclude_cols = ['Air Temperature', 'CO2', 'Humidity']

# Select all columns except the excluded ones
df_filtered = extract.drop(columns=exclude_cols)

df_filtered.head()

sensor,dbtimestamp,devicename,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH,NPK Ratio,N Category,P Category,K Category,NPK Level,NPK Done
0,2025-05-30 00:01:55.980744,NDS016,213.0,18.9,6.0,59.0,52.0,25.6,5.5,1.0 : 9.8 : 8.7,Low,Low,Low,Invalid,Not Done Composting
1,2025-05-29 00:01:26.817793,NDS016,240.0,18.9,11.0,72.0,65.0,27.8,5.5,1.0 : 6.5 : 5.9,Low,Low,Low,Invalid,Not Done Composting
2,2025-05-28 00:00:48.939404,NDS016,247.0,18.1,13.0,76.0,68.0,26.5,5.5,1.0 : 5.8 : 5.2,Low,Low,Low,Invalid,Not Done Composting
3,2025-05-27 00:00:19.025437,NDS016,270.0,19.2,17.0,87.0,80.0,27.3,5.5,1.0 : 5.1 : 4.7,Low,Low,Low,Invalid,Not Done Composting
4,2025-05-26 00:01:19.007408,NDS016,292.0,20.7,22.0,98.0,90.0,26.9,5.2,1.0 : 4.5 : 4.1,Low,Low,Low,Invalid,Not Done Composting


In [31]:
# Apply value filters
df_filtered = df_filtered[
    (df_filtered['Soil EC'].between(0, 3000)) &
    (df_filtered['Soil Moisture'].between(0, 3000)) &
    (df_filtered['Soil Nitrogen'].between(0, 3000)) &
    (df_filtered['Soil Phosphorus'].between(0, 3000)) &
    (df_filtered['Soil Potassium'].between(0, 3000)) &
    (df_filtered['Soil Temperature'].between(20, 35)) &
    (df_filtered['Soil pH'].between(1, 12))
]

df_filtered.head()


sensor,dbtimestamp,devicename,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH,NPK Ratio,N Category,P Category,K Category,NPK Level,NPK Done
0,2025-05-30 00:01:55.980744,NDS016,213.0,18.9,6.0,59.0,52.0,25.6,5.5,1.0 : 9.8 : 8.7,Low,Low,Low,Invalid,Not Done Composting
1,2025-05-29 00:01:26.817793,NDS016,240.0,18.9,11.0,72.0,65.0,27.8,5.5,1.0 : 6.5 : 5.9,Low,Low,Low,Invalid,Not Done Composting
2,2025-05-28 00:00:48.939404,NDS016,247.0,18.1,13.0,76.0,68.0,26.5,5.5,1.0 : 5.8 : 5.2,Low,Low,Low,Invalid,Not Done Composting
3,2025-05-27 00:00:19.025437,NDS016,270.0,19.2,17.0,87.0,80.0,27.3,5.5,1.0 : 5.1 : 4.7,Low,Low,Low,Invalid,Not Done Composting
4,2025-05-26 00:01:19.007408,NDS016,292.0,20.7,22.0,98.0,90.0,26.9,5.2,1.0 : 4.5 : 4.1,Low,Low,Low,Invalid,Not Done Composting


In [32]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 722 entries, 0 to 1000
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   dbtimestamp       722 non-null    datetime64[ns]
 1   devicename        722 non-null    object        
 2   Soil EC           722 non-null    float64       
 3   Soil Moisture     722 non-null    float64       
 4   Soil Nitrogen     722 non-null    float64       
 5   Soil Phosphorus   722 non-null    float64       
 6   Soil Potassium    722 non-null    float64       
 7   Soil Temperature  722 non-null    float64       
 8   Soil pH           722 non-null    float64       
 9   NPK Ratio         722 non-null    object        
 10  N Category        722 non-null    object        
 11  P Category        722 non-null    object        
 12  K Category        722 non-null    object        
 13  NPK Level         722 non-null    object        
 14  NPK Done          722 non-null

In [33]:
# Will not be using NDS002 for now

df_filtered.groupby('devicename').size()

devicename
NDS002      1
NDS004     98
NDS005     98
NDS006     78
NDS011     98
NDS012     98
NDS015     78
NDS016    173
dtype: int64

Connnect to NP Feeding Log

In [34]:
# Load the uploaded Excel file
excel_path = "[NgeeAnn] Feeding Logs.xlsx"

In [35]:
import pandas as pd

# Load Excel file
xls = pd.ExcelFile(excel_path)
sheet_names = xls.sheet_names

# Load the 3rd and 4th sheets (indexes 2 and 3)
df_third = xls.parse(sheet_names[2])
df_fourth = xls.parse(sheet_names[3])
df_fifth = xls.parse(sheet_names[4])

# Concatenate them (by default, axis=0, i.e., stack rows)
df_feeding_log = pd.concat([df_third, df_fourth, df_fifth], ignore_index=True)

# Preview the result
df_feeding_log.head()

,Date,NDS001,NDS002,NDS003,NDS004,NDS005,NDS006,NDS007,NDS008,NDS009,NDS010,NDS011,NDS012,NDS013,NDS014,NDS015,NDS016
0,2024-06-01,"70g of distilled water, 3g of dried radish top...","4g of fresh mint leaves, 5g of dried mango ski...","4g of dried radish tops, 6g of crushed eggshel...","50g of distilled water, 1g of dried radish top...","50g of distilled water, 6g of crushed eggshell...","3g of dried radish tops, 5g of dried mango skins","15g of coffee grounds, 3g of dried chye sim","3g of dried radish tops, 7g of crushed eggshel...","25g of coffee grounds, 4g of dried chye sim","5g of dried radish tops, 6g of crushed eggshel...","45g of distilled water, 5g of dried radish top...",NaN,NaN,NaN,NaN,NaN
1,2024-06-02,45g of distilled water,"45g of distilled water, 4g of dried chye sim","3g of dried radish tops, 8g of crushed eggshel...","8g of crushed eggshells, 10g of coffee grounds...","70g of distilled water, 15g of coffee grounds","40g of distilled water, 3g of dried radish top...","5g of dried radish tops, 15g of coffee grounds...","40g of distilled water, 4g of crushed eggshell...",45g of distilled water,2g of dried mango skins,"100g of distilled water, 5g of crushed eggshel...","50g of dried radish tops, 50g of dried chye sim","50g of dried radish tops, 50g of dried chye sim","50g of dried radish tops, 50g of dried chye sim",NaN,"50g of dried radish tops, 50g of dried chye sim"
2,2024-06-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-06-04,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water
4,2024-06-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Filter Out Tanks:

1) NDS002      1
2) NDS004     98
3) NDS005     98
4) NDS006     98
5) NDS011     98
6) NDS012     98
7) NDS015     98
8) NDS016    172

In [36]:
df_feeding_log = df_feeding_log[['Date', 'NDS004', 'NDS005', 'NDS006', 'NDS011', 'NDS012', 'NDS015', 'NDS016']]

df_feeding_log.head()

,Date,NDS004,NDS005,NDS006,NDS011,NDS012,NDS015,NDS016
0,2024-06-01,"50g of distilled water, 1g of dried radish top...","50g of distilled water, 6g of crushed eggshell...","3g of dried radish tops, 5g of dried mango skins","45g of distilled water, 5g of dried radish top...",NaN,NaN,NaN
1,2024-06-02,"8g of crushed eggshells, 10g of coffee grounds...","70g of distilled water, 15g of coffee grounds","40g of distilled water, 3g of dried radish top...","100g of distilled water, 5g of crushed eggshel...","50g of dried radish tops, 50g of dried chye sim",NaN,"50g of dried radish tops, 50g of dried chye sim"
2,2024-06-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-06-04,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water,200g of water
4,2024-06-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
convert_feedlog = df_feeding_log.melt(
    id_vars='Date',
    var_name='devicename',
    value_name='feeding_description'
)

# Preview the transformed feeding log
convert_feedlog.head()

,Date,devicename,feeding_description
0,2024-06-01,NDS004,"50g of distilled water, 1g of dried radish top..."
1,2024-06-02,NDS004,"8g of crushed eggshells, 10g of coffee grounds..."
2,2024-06-03,NDS004,NaN
3,2024-06-04,NDS004,200g of water
4,2024-06-05,NDS004,NaN


Need to seperate the feed log to get exact amount of green or browns

In [38]:
convert_feedlog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2128 entries, 0 to 2127
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Date                 2128 non-null   datetime64[ns]
 1   devicename           2128 non-null   object        
 2   feeding_description  427 non-null    object        
dtypes: datetime64[ns](1), object(2)
memory usage: 50.0+ KB


Green & Brown Categorization

In [39]:
# Load the first sheet, which contains the greens and browns reference list
df_greens_browns = pd.read_excel(excel_path, sheet_name=0, header=1)

# Display to inspect
df_greens_browns.head()

,S/N,Food Name,Emissions (kg CO2/kg),Category,pH Value
0,1,Bread crusts,0.14,Brown,Neutral
1,2,Cauliflower leaves,0.07,Brown,Acidic
2,3,Cheese rinds,1.20,Brown,Acidic
3,4,Coffee grounds,0.20,Brown,Acidic
4,5,Ground coffee,0.23,Brown,Acidic


In [40]:
df_greens_browns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   S/N                    84 non-null     int64  
 1   Food Name              84 non-null     object 
 2   Emissions (kg CO2/kg)  84 non-null     float64
 3   Category               84 non-null     object 
 4   pH Value               84 non-null     object 
dtypes: float64(1), int64(1), object(3)
memory usage: 3.4+ KB


Add brown & green categorization, feeding description

In [41]:
# Step 2: Normalize food name reference
df_greens_browns['Food Name'] = df_greens_browns['Food Name'].str.lower().str.strip()
df_greens_browns['Category'] = df_greens_browns['Category'].str.strip()
food_to_category = dict(zip(df_greens_browns['Food Name'], df_greens_browns['Category']))

In [42]:
# Step 3: Function to classify feeding description
def categorize_feeding(desc):
    if pd.isnull(desc):
        return 'None'
    desc = desc.lower()
    matches = [food_to_category[food] for food in food_to_category if food in desc]
    if not matches:
        return 'Unknown'
    return matches[0] if all(x == matches[0] for x in matches) else 'Mixed'

In [43]:
# Step 4: Apply feeding category classification to feeding log
convert_feedlog['Feeding Category'] = convert_feedlog['feeding_description'].apply(categorize_feeding)

In [44]:
convert_feedlog.head()

,Date,devicename,feeding_description,Feeding Category
0,2024-06-01,NDS004,"50g of distilled water, 1g of dried radish top...",Green
1,2024-06-02,NDS004,"8g of crushed eggshells, 10g of coffee grounds...",Mixed
2,2024-06-03,NDS004,NaN,None
3,2024-06-04,NDS004,200g of water,Unknown
4,2024-06-05,NDS004,NaN,None


In [45]:
# Renaming columns
convert_feedlog.rename(columns={'Date': 'dbtimestamp'}, inplace=True)
convert_feedlog.head()

,dbtimestamp,devicename,feeding_description,Feeding Category
0,2024-06-01,NDS004,"50g of distilled water, 1g of dried radish top...",Green
1,2024-06-02,NDS004,"8g of crushed eggshells, 10g of coffee grounds...",Mixed
2,2024-06-03,NDS004,NaN,None
3,2024-06-04,NDS004,200g of water,Unknown
4,2024-06-05,NDS004,NaN,None


In [46]:
df_combined = pd.merge(
    df_filtered.assign(date=pd.to_datetime(df_filtered['dbtimestamp']).dt.date),
    convert_feedlog.assign(date=pd.to_datetime(convert_feedlog['dbtimestamp']).dt.date)[['devicename', 'date', 'feeding_description', 'Feeding Category']],
    on=['devicename', 'date'],
    how='left'
)

In [47]:
df_combined = df_combined.drop('date', axis=1)
df_combined.head()

,dbtimestamp,devicename,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH,NPK Ratio,N Category,P Category,K Category,NPK Level,NPK Done,feeding_description,Feeding Category
0,2025-05-30 00:01:55.980744,NDS016,213.0,18.9,6.0,59.0,52.0,25.6,5.5,1.0 : 9.8 : 8.7,Low,Low,Low,Invalid,Not Done Composting,NaN,NaN
1,2025-05-29 00:01:26.817793,NDS016,240.0,18.9,11.0,72.0,65.0,27.8,5.5,1.0 : 6.5 : 5.9,Low,Low,Low,Invalid,Not Done Composting,NaN,NaN
2,2025-05-28 00:00:48.939404,NDS016,247.0,18.1,13.0,76.0,68.0,26.5,5.5,1.0 : 5.8 : 5.2,Low,Low,Low,Invalid,Not Done Composting,NaN,NaN
3,2025-05-27 00:00:19.025437,NDS016,270.0,19.2,17.0,87.0,80.0,27.3,5.5,1.0 : 5.1 : 4.7,Low,Low,Low,Invalid,Not Done Composting,NaN,NaN
4,2025-05-26 00:01:19.007408,NDS016,292.0,20.7,22.0,98.0,90.0,26.9,5.2,1.0 : 4.5 : 4.1,Low,Low,Low,Invalid,Not Done Composting,NaN,NaN


Extract Data

In [48]:
df_combined.to_csv("2024 Extracted.csv ", index=False)